In [11]:
import os
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
msg_path = '/home/anna.jungbluth/metrics/eo-miniset/msg'
goes_path = '/home/anna.jungbluth/metrics/eo-miniset/goes'

msg_files = [os.path.join(msg_path, f) for f in os.listdir(msg_path) if f.endswith('.csv')]
msg_files.sort()
goes_files = [os.path.join(goes_path, f) for f in os.listdir(goes_path) if f.endswith('.csv')]
goes_files.sort()

In [22]:
def parse_numpy_float64_string(s):
    # Remove brackets and numpy function calls
    clean = s.strip('[]').replace('np.float64(', '').replace(')', '')
    # Split on commas and convert to floats
    return [float(x.strip()) for x in clean.split(',')]

def parse_numpy_float32_string(s):
    # Remove brackets and numpy function calls
    clean = s.strip('[]').replace('np.float32(', '').replace(')', '')
    # Split on commas and convert to floats
    return [float(x.strip()) for x in clean.split(',')]

In [23]:
dataframes = []

for msg_file in msg_files:
    msg_df = pd.read_csv(msg_file)

    # Convert string representations of numpy arrays to lists
    for column in msg_df.columns:
        if msg_df[column].dtype == 'object':
            try:
                msg_df[column] = msg_df[column].apply(parse_numpy_float64_string)
            except ValueError:
                msg_df[column] = msg_df[column].apply(parse_numpy_float32_string)
            else:
                pass


    dataframes.append(msg_df)

master_df = pd.concat(dataframes, ignore_index=True)
master_df.to_csv('./raw-summary-files/msg_2020_miniset.csv', index=False)

In [24]:
dataframes = []

for goes_file in goes_files:
    goes_df = pd.read_csv(goes_file)
    # Convert string representations of numpy arrays to lists
    for column in goes_df.columns:
        if goes_df[column].dtype == 'object':
            try:
                goes_df[column] = goes_df[column].apply(parse_numpy_float64_string)
            except ValueError:
                goes_df[column] = goes_df[column].apply(parse_numpy_float32_string)
            else:
                pass
    dataframes.append(goes_df)


master_df = pd.concat(dataframes, ignore_index=True)
# Convert numpy float types to native Python floats
master_df = master_df.map(lambda x: float(x) if isinstance(x, np.floating) else x)

master_df.to_csv('./raw-summary-files/goes_2020_miniset.csv', index=False)